### The logic of the notebook

In order to be able to perform multiple operations I have designed the system as follows:

- first, I create the most basic function which just updates the owner of a player
- then I defined the class transfer, with argument name of the player
    - the class transfer can perform all market operations and then save database entries
    - however, it is not convenient to save directly to the database, in case there are accidental mistakes
- so I created another class called market session:
    - first, you start a market session and create each transfer within the session using the inner class transfer
    - when you are done with all the transfers, and are sure there are no mistakes you confirm the session
    - if you notice a mistake you can cancel the session and the databases rest unchanged
    
    
## OPEN QUESTIONS:

- what is the cost for draft? I set it to 0 for now
- is this the best way to define exchanges?
- we could in principle include in the player's info last transfer operation's id, but it is enough to look at the most recent in the transfer db

In [52]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
import json
import copy
from pymongo import MongoClient
from pprint import pprint
import pymongo
import datetime
import ipywidgets as widgets
from IPython.display import HTML, clear_output

with open('credential.json','r') as f:
    cred = json.load(f)

cluster = MongoClient(cred['cred'])
# choosing database
db = cluster["Game"]
# choosing collection
collection = db["Players"]

In [2]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [53]:
"""
Function documentation:
This is the general function to update the owner of a player.
Arguments:

- new_owner: string, the new owner of the team (pietro, luca,...)
- mode: string, the acquisition mode (Scambio, draft,...)
- squad: assignment in the new team (main or primavera)
- name/id: string, use one OR the other as a filter to query the database
- time_stamp: boolean. True if you want to register the date of operation in the database
- rest: details to access database, set to default values

Note: I'm planning to use this function inside higher level functions, which is why I have the timestamp option, 
if you are doing tests there is no reason to update the date, but if you include this in a market operation function
(for instance exchange players) you want to register this and the higher level function will also create the database entry

"""

def update_owner(new_owner, mode, squad, loan = False, move_up = False, name = False, Id = False, time_stamp = False, CR = cred['cred'], DB = 'Game', CO = 'Players'):
    
    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[CO]
    
    today = date.today()
    d1 = today.strftime("%Y/%m/%d")
    
    if name:
        filt = {'name': name}
    elif Id:
        filt = {'_id': str(Id)}
    else:
        print('Please enter name or Id argument')
        
        return None
        
    dic = collection.find_one(filt)
    old_owner = dic['info']['contract']['owner'] #when database structure changes, Modify this and following accordingly
    
    if dic['info']['current_team']['owner'] is None:
        old_team = None
    else:
        old_team = dic['info']['current_team']['owner'] +', ' + dic['info']['current_team']['squad']
    
    if loan:

        collection.update_one(filt, {'$set':{'info.current_team.previous_team': old_team}}) 
        collection.update_one(filt, {'$set':{'info.current_team.owner': new_owner}})
        collection.update_one(filt, {'$set':{'info.current_team.squad': squad}})
        
        
        if True:
            collection.update_one(filt, {'$set':{'info.current_team.start_date': d1}}) #
            #print("Operation date: ")
            return d1
    
    elif move_up:
        
        collection.update_one(filt, {'$set':{'info.current_team.previous_team': old_team}}) 
        collection.update_one(filt, {'$set':{'info.current_team.squad': squad}})
        
        if True:
            collection.update_one(filt, {'$set':{'info.current_team.start_date': d1}}) #
            #print("Operation date: ")
            return d1
        
        
    else:
    
        collection.update_one(filt, {'$set':{'info.contract.acquisition_mode': mode}})
        collection.update_one(filt, {'$set':{'info.contract.previous_owner': old_owner}})
        collection.update_one(filt, {'$set':{'info.contract.owner': new_owner}})

        collection.update_one(filt, {'$set':{'info.current_team.previous_team': old_team}}) 
        collection.update_one(filt, {'$set':{'info.current_team.owner': new_owner}})
        collection.update_one(filt, {'$set':{'info.current_team.squad': squad}})
    
        if True:
            collection.update_one(filt, {'$set':{'info.contract.start_date': d1}})
            collection.update_one(filt, {'$set':{'info.current_team.start_date': d1}}) #redundant
            #print("Operation date: ")
            return d1

In [54]:
def update_Tr_DB(transfer_data, CR = cred['cred'], DB = 'Game', COT = 'Transfers'):
    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[COT]
    
    now = datetime.datetime.today()
    Id = now.strftime("%Y%m%d%H%M%S")
    #d1 = now.strftime("%Y/%m/%d")
    
    dic0 = transfer_data[0]
    
    if dic0['operation'] in ['Asta', 'Draft', 'Acquisto', 'Prestito', 'Fine Prestito', 'Promozione']:
        dic0['_id'] = Id
        #dic0['date'] = d1
        
        collection.insert_one(dic0)
    
    if dic0['operation'] == 'Scambio':
        
        dic1 = transfer_data[1]
        
        
        dic0['_id'] = Id
        #dic0['date'] = d1
        #dic1['date'] = d1
        dic1['_id'] = str(int(Id) + 1)
        
        collection.insert_one(dic0)
        collection.insert_one(dic1)
    

In [55]:
"""
This is an inner level class, it performs all the market operations and creates database entries
But we'll use it inside another class, so all operations need to be confirmed before being saved

"""


class transfer:
    
    #need to add quotation info everywhere, maybe as return of transf_db

    def __init__(self, name, CR, DB, CO, COT, CO_Man):
        
        cluster = MongoClient(CR)
        db = cluster[DB]
        
        self.collection = db[CO]
        self.collection_man = db[CO_Man]
        self.name = name
        self.info = collection.find_one({'name': name})
        self.CC = (CR, DB, CO)
        self.COT = COT
        
        
        
        today = date.today()
        self.d1 = today.strftime("%Y/%m/%d")
         
        
    def draft_in(self, new_owner, squad, cost = 0):
        (CR, DB, CO) = self.CC
        
        collection_man = self.collection_man
        man_data = collection_man.find_one({'owner': new_owner})
        print(new_owner)
        budget = man_data['budget']
        
        if cost > budget:
            return False
        else:
            
            #print(new_owner+' has acquired '+self.name+' through draft.\nAssigned to '+squad+' squad\n')


            update_owner(new_owner = new_owner, mode = 'Draft', squad = squad, name = self.name, time_stamp = False, CR = CR, DB = DB, CO = CO)

            self.collection.update_one({'name': self.name},{'$set':{'info.contract.cost': cost}})


            transfer_data = [{
                'name': self.name,
                'operation': 'Draft',
                'new_owner': new_owner,
                'squad': squad,
                'cost': cost,
                'date': self.d1
                          }]


            update_Tr_DB(transfer_data, CR, DB, self.COT)
            self.collection_man.update_one({'owner': new_owner}, {'$inc': {'budget': - cost}})
            #transf_db_update() #function to be defined
        
        return True
        
        
    
    def auction_in(self, new_owner, squad, cost):
        (CR, DB, CO) = self.CC
        #print(new_owner+' has acquired '+self.name+' through auction for '+str(cost)+' FM.\nAssigned to '+squad+' squad\n')
        update_owner(new_owner = new_owner, mode = 'Asta', squad = squad, name = self.name, time_stamp = False, CR = CR, DB = DB, CO = CO)
        
        self.collection.update_one({'name': self.name},{'$set':{'info.contract.cost': cost}})
        #transf_db_update() #function to be defined, includes budgets adjustments
        
        transfer_data = [{
            'name': self.name,
            'operation': 'Asta',
            'new_owner': new_owner,
            'squad': squad,
            'cost': cost,
            'date': self.d1
                      }]
        
        
        update_Tr_DB(transfer_data, CR, DB, self.COT)
        
        
        
    
    def cash_in(self, new_owner, squad, cost):
        old_owner = self.info['info']['contract']['owner']
        (CR, DB, CO) = self.CC
        #print(new_owner+' has acquired '+self.name+' through auction for '+str(cost)+' FM.\nAssigned to '+squad+' squad\n')
        update_owner(new_owner = new_owner, mode = 'Acquisto', squad = squad, name = self.name, time_stamp = False, CR = CR, DB = DB, CO = CO)
        
        self.collection.update_one({'name': self.name},{'$set':{'info.contract.cost': cost}})
        #transf_db_update() #function to be defined, includes budgets adjustments
        
        transfer_data = [{
            'name': self.name,
            'operation': 'Acquisto',
            'new_owner': new_owner,
            'old_owner': old_owner,
            'squad': squad,
            'cost': cost,
            'date': self.d1
                      }]
        
        
        update_Tr_DB(transfer_data, CR, DB, self.COT)
        
        
    
    def exchange_with(self, exch_player, cost, squad, squad_exch_pl):
        
        #THIS IS PROBABLY THE MOST PROBLEMATIC, MAYBE WE NEED A BETTER WAY TO DEFINE IT
        #MAYBE JUST RENAMING SOME VARIABLES WILL SUFFICE
        
        (CR, DB, CO) = self.CC
        collection = self.collection
        self.exch_player = exch_player
        
        
        exch_info = collection.find_one({'name': exch_player})
        exch_owner = exch_info['info']['contract']['owner']
        owner = self.info['info']['contract']['owner']
        
        
        d1 = update_owner(new_owner = exch_owner, mode = 'Scambio', squad = squad, name = self.name, time_stamp = False, CR = CR, DB = DB, CO = CO)
        d1 = update_owner(new_owner = owner, mode = 'Scambio', squad = squad_exch_pl, name = exch_player, time_stamp = False, CR = CR, DB = DB, CO = CO)
        
        self.collection.update_one({'name': self.name},{'$set':{'info.contract.cost': max(0, cost)}})
        self.collection.update_one({'name': exch_player},{'$set':{'info.contract.cost': max(0, - cost)}})
        
        
        transfer_data_0 = {
            'name': self.name,
            'operation': 'Scambio',
            'new_owner': exch_owner,
            'exchange_player': exch_player,
            'squad': squad,
            'cost': max(0, cost),
            'date': self.d1
                      }
        transfer_data_1 = {
            'name': exch_player,
            'operation': 'Scambio',
            'new_owner': owner,
            'exchange_player': self.name,
            'squad': squad_exch_pl,
            'cost': max(0, -cost),
            'date': self.d1
                      }
        transfer_data = [transfer_data_0, transfer_data_1]
        
        update_Tr_DB(transfer_data, CR, DB, self.COT)
        
        
        
    
    def loan_to(self, owner, cost, squad, end_season = None, final_fee = None):
        (CR, DB, CO) = self.CC
        collection = self.collection
        update_owner(new_owner = owner, mode = None, squad = squad, loan = True, name = self.name, time_stamp = False, CR = CR, DB = DB, CO = CO)
        
        
        date = datetime.date.today()
        
        m, y = date.month, date.year
        if m > 7:
            y = y + 1
        
        loan_info = {
        'start_date': self.d1,
        'expire_date': str(y)+'/07/31', #all loans expire the successive July 31st
        'cost': cost,
        'redemption_info': end_season, #None, Option or Must
        'redemption_cost': final_fee, # None or >0
        }
        
        collection.update_one({'name': self.name}, {'$set':{'info.current_team.on_loan': True}}) 
        collection.update_one({'name': self.name}, {'$set':{'info.current_team.loan_info': loan_info}}) 
        
        transfer_data = [{
            'name': self.name,
            'operation': 'Prestito',
            'new_team': owner +', '+ squad,
            'loan_info': loan_info,
            'date': self.d1
                      }]
        
        
        update_Tr_DB(transfer_data, CR, DB, self.COT)
        
        
    
    def end_loan(self, squad, redeemed = False):
        (CR, DB, CO) = self.CC
        collection = self.collection
        
        dic = self.info
        owner = dic['info']['contract']['owner']
        
        update_owner(new_owner = owner, mode = None, squad = squad, loan = True, name = self.name, time_stamp = False, CR = CR, DB = DB, CO = CO)
        
        collection.update_one({'name': self.name}, {'$set':{'info.current_team.on_loan': False}}) 
        collection.update_one({'name': self.name}, {'$set':{'info.current_team.loan_info': None}}) 
        
        transfer_data = [{
            'name': self.name,
            'operation': 'Fine Prestito',
            'new_team': owner +', '+ squad,
            'redeemed': redeemed,
            'date': self.d1
                      }]
        
        
        update_Tr_DB(transfer_data, CR, DB, self.COT)
        
        if redeemed:
            
            print('Registrare operazione di trasferimento!!!')
        
        
        
    def move_up(self):
        (CR, DB, CO) = self.CC
        update_owner(new_owner = None, mode = None, squad = 'Main', move_up = True, name = self.name, time_stamp = False, CR = CR, DB = DB, CO = CO)
        
        transfer_data = [{
            'name': self.name,
            'operation': 'Promozione',
            'squad': 'Main',
            'date': self.d1
                      }]
        
        
        update_Tr_DB(transfer_data, CR, DB, self.COT)
        
        return d1
        
        

In [56]:
"""
This second layer is designed to prevent accidental changes in the database:
First all the players collection is duplicated to a temp one
then all the changes are made in the temp

NOTE: we need to add the equivalent for the transfer database, but we first need to design the db_update function
"""

class market_session:
    
    def __init__(self, ID, CR = cred['cred'], DB = 'Game', CO = 'Players', COT = 'Transfers', CO_MAN = 'Managers'):
        #copy entire database into tempPlayers
        
        cluster = MongoClient(CR)
        db = cluster[DB]
        collection = db[CO]
        collection_man = db[CO_MAN]
        COt = 'tempPlayers'
        CO_MANt = 'tempManagers'
        
        collectionT = db[COt]
        collection_manT = db[CO_MANt]
        
        posts = list(collection.find({}))
        collectionT.insert_many(posts)
        
        posts_man = list(collection_man.find({}))
        collection_manT.insert_many(posts_man)
        
        self.ID = ID
        self.CC = (CR, DB, COt)
        self.CO = CO
        self.COTt = 'tempTransfers'
        self.CO_MANt = 'tempManagers'
        
    def select_pl(self, name):
        (CR, DB, COt) = self.CC
        self.player = transfer(name, CR, DB, COt, self.COTt, self.CO_MANt)
    
    def confirm_session(self):
        #here we span through both temp collections (players and transfers) and replace/add entries to the official database
        #then we erase the temp databases
        (CR, DB, COt) = self.CC
        CO = self.CO
        
        cluster = MongoClient(CR)
        db = cluster[DB]
        collection = db[CO]
        collectionT = db[COt]
        posts = list(collectionT.find({}))
        collection.delete_many({})
        collection.insert_many(posts)
        collectionT.delete_many({})
        
        coll_tr = db[COT]
        coll_trT = db[self.COTt]
        posts = list(coll_trT.find({}))
        coll_tr.delete_many({})
        coll_tr.insert_many(posts)
        coll_trT.delete_many({})
        
        coll_man = db[CO_MAN]
        coll_manT = db[self.CO_MANt]
        posts = list(coll_manT.find({}))
        coll_man.delete_many({})
        coll_man.insert_many(posts)
        coll_manT.delete_many({})
        
        
    def cancel_session(self):
        #erase temporary database: need to redefine market session
        (CR, DB, COt) = self.CC
        
        cluster = MongoClient(CR)
        db = cluster[DB]
        collectionT = db[COt]
        coll_trT = db[self.COTt]
        coll_manT = db[self.CO_MANt]
        
        collectionT.delete_many({})
        coll_trT.delete_many({})
        coll_manT.delete_many({})

In [57]:
def player(name, session):
    #select one player and assign it to a variable 
    session.select_pl(name)
    return session.player #returns a class transfer object

In [9]:
#define market session with id number (not necessary)
#sess = market_session(1)

In [10]:
#create a transfer class object
musso = player('MUSSO', sess)

NameError: name 'sess' is not defined

In [10]:
#other examples
ronaldo = player('RONALDO', sess)
lukaku = player('LUKAKU', sess)
ibra = player('IBRAHIMOVIC', sess)
immobile = player('IMMOBILE', sess)
orso = player('ORSOLINI', sess)

In [11]:
#check name
musso.name

'MUSSO'

In [12]:
#example draft
musso.draft_in('luca', squad = 'primavera')

Operation date: 


'2020/12/23'

In [13]:
#example auction
ronaldo.auction_in('enzo', squad = 'main', cost = 60)

Operation date: 


'2020/12/23'

In [14]:
#example buy cash
lukaku.cash_in(new_owner = 'nanni', squad = 'main', cost = 100)

Operation date: 


'2020/12/23'

In [15]:
#example of exchange operation. The point of view is of the receiving owner:
#the cost paid by whoever receives the exchange pl (ronaldo here)
#therefore, in this case, if IBRA's NEW owner pays, cost>0, viceversa, cost<0. squad is IBRA's future squad
ibra.exchange_with('MARTINEZ L.', cost = 4, squad = 'primavera', squad_exch_pl = 'main')

Operation date: 


'2020/12/23'

In [16]:
#example loan
immobile.loan_to(owner = 'luca', cost = 25, squad = 'main', end_season = 'Option', final_fee = 30)

Operation date: 


'2020/12/23'

In [17]:
#example end loan (note that in the same session this might cause problems, but it should never happen)
immobile.end_loan(squad = 'main', redeemed = True)

Operation date: 
Registrare operazione di trasferimento!!!


'2020/12/23'

In [18]:
#example buying after loan
immobile.cash_in(new_owner = 'luca', squad = 'main', cost = 30)

Operation date: 


'2020/12/23'

In [19]:
#example promotion
orso.move_up()

Operation date: 


'2020/12/23'

In [42]:
#CONFIRM: RUN ONLY IF TRANSFERS ARE FINAL

#sess.confirm_session()

#CANCEL: RUN TO DISCARD ALL THE TRANSFERS IN THE DEFINED SESSION

sess.cancel_session()

In [47]:
db['tempTransfers'].delete_many({})
db['tempPlayers'].delete_many({})
db['tempManagers'].delete_many({})

## Now I'll try to set this up using ipython widgets

In [7]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

### Session start/cancel/confirm
Run Start it at the beginning and Confirm at the end
If some error is made you can cancel the entire session by clicking Cancel.

Note, the funds control is finalized only for draft operations so far, use the same logic to enforce it in the others:
- Make operations perform transfers only when funds are available
- return True/False if it went through
- use the return value to display message

In [58]:
btn_start = widgets.Button(description='Start Session',layout=widgets.Layout(width='200px', height='40px'))

btn_cancel = widgets.Button(description='Cancel Session',layout=btn_start.layout)

btn_confirm = widgets.Button(description='Confirm Session',layout=btn_start.layout)

def sess_start_event(obj):
    global sess
    print('Session Started!', end="\r", flush=True)#.format(obj.description))
    sess = market_session(1)
def sess_cancel_event(obj):
    print('Session Canceled!', end="\r", flush=True)#.format(obj.description))
    sess.cancel_session()
def sess_confirm_event(obj):
    print('Session Confirmed!', end="\r", flush=True)#.format(obj.description))
    sess.cancel_session()
    
    
btn_start.on_click(sess_start_event)
btn_start.style.button_color = 'skyblue'
btn_cancel.on_click(sess_cancel_event)
btn_cancel.style.button_color = 'lightcoral'  
btn_confirm.on_click(sess_confirm_event)
btn_confirm.style.button_color = 'lightgreen'   

input_start = widgets.HBox([btn_start, btn_cancel, btn_confirm])
display(input_start)


### Selection of the  player to be transfered
RUN THE CELL ONLY AFTER SESSION START
Note: Players are listed by OWNER, so those on loan are not under the team they play with

In [59]:
owners = ['Choose one','Svincolati','luca','pietro','enzo','nanni','mario','emiliano','franky','musci8']


def owner_players(owner):
    if owner == 'Svincolati':
        results = db['tempPlayers'].find({'info.contract.owner': None})
        names = []
        for result in results:
            names.append(result['name'])
    elif owner == 'Choose one':
        names = ['Choose owner first']
    else:
        results = db['tempPlayers'].find({'info.contract.owner': owner})
        names = []
        for result in results:
            names.append(result['name'])
    names.sort()
    return names

def confirm_player(Name):
    global pl
    pl = player(Name, sess)
    if Name == 'Choose owner first':
        print('')
    else:
        print('The player selected to be transferred is ' + Name+'                     ')
    

def selection(pl_owner):
    global G_owner
    G_owner = pl_owner
    lista.options = owner_players(pl_owner)



scW = widgets.Dropdown(options=owners, description = 'Owner')

init = scW.value

lista = widgets.Dropdown(options=owner_players(init), description = 'Player')

j = widgets.interactive(confirm_player, Name=lista)

i = widgets.interactive(selection, pl_owner=scW)





# new_owner, squad, cost, exch_player, squad_exch_pl, end_season, final_fee


#display(i)
#display(j)

btn_head_pl = widgets.Button(description='Select Player to Transfer',layout=widgets.Layout(width='350px', height='40px'))

btn_head_pl.style.button_color = 'skyblue'


input_pl = widgets.VBox([btn_head_pl,i,j])
display(input_pl)


### Selection of parameters


Use only the ones that are relevant to the market operation

In [60]:
cost = 0
New_owner = widgets.Dropdown(options =    owners, description = 'New Owner')

def new_owner_eventhandler(change):
    global new_owner
    new_owner = change.new
    print('After operation '+pl.name+' will be owned by ' + new_owner+'                               ', end="\r", flush=True)

New_owner.observe(new_owner_eventhandler, names = 'value')

#display(New_owner)

squads = widgets.Dropdown(options = ['Choose', 'Main', 'Primavera'], description = 'New Squad')

def squad_eventhandler(change):
    global squad
    squad = change.new
    print('After operation '+pl.name+' will be in ' + new_owner+'\'s '+squad+' squad                       ', end="\r", flush=True)

squads.observe(squad_eventhandler, names = 'value')

#display(squads)

cost_num = widgets.BoundedFloatText(min=-1000, max=1000, value=0, step=1, description = 'Cost')

def cost_num_eventhandler(change):
    global cost
    cost = change.new
    if cost >= 0:
        print(new_owner+' will pay ' + str(change.new)+' FM for the operation involving '+pl.name+'                              ', end="\r", flush=True)
    else:
        print(new_owner+' will receive ' + str(- change.new)+' FM for the operation involving '+pl.name+'                          ', end="\r", flush=True)
        
cost_num.observe(cost_num_eventhandler, names='value')


End_season = widgets.Dropdown(options = ['Choose', None, 'Option', 'Must'], description = 'End Choice')

def End_season_eventhandler(change):
    global end_season
    end_season = change.new
    if end_season is None:
        print('If loan is chosen, no option is selected for end of the season                                         ', end="\r", flush=True)
    elif end_season is 'Option':
        print('If loan is chosen, '+pl.name+' CAN be redeemed at final fee value                                     ', end="\r", flush=True)
    else:
        print('If loan is chosen, '+pl.name+' MUST be redeemed at final fee value                                    ', end="\r", flush=True)

End_season.observe(End_season_eventhandler, names = 'value')



fee_num = widgets.BoundedFloatText(min=-1000, max=1000, value=0, step=1, description = 'Final fee')

def fee_num_eventhandler(change):
    global final_fee
    final_fee = change.new
    if cost >= 0:
        print(new_owner+' can/will pay ' + str(final_fee)+' to redeem '+pl.name+' at the end of the season              ', end="\r", flush=True)
        
fee_num.observe(fee_num_eventhandler, names='value')

Redeemed = widgets.Dropdown(options = ['Choose', True,False], description = 'Redeemed')

def End_loan_eventhandler(change):
    global redeemed
    redeemed = change.new
    if redeemed == True:
        print('If loan is chosen, '+pl.name+' will be redeemed.                 ', end="\r", flush=True)
    else:
        print('If loan is chosen, '+pl.name+' will not be redeemed.                  ', end="\r", flush=True)

Redeemed.observe(End_loan_eventhandler, names = 'value')


#display(New_owner)
#display(squads)
#display(cost_num)

#display(End_season)
#display(fee_num)
#display(Redeemed)

btn_head_par = widgets.Button(description='Generic Parameters',layout=widgets.Layout(width='300px', height='40px'))
btn_head_par_loan = widgets.Button(description='Loan Parameters',layout = btn_head_par.layout)
btn_head_par.style.button_color = 'skyblue'
btn_head_par_loan.style.button_color = 'skyblue'



left_box = widgets.VBox([btn_head_par,New_owner, squads, cost_num])
right_box = widgets.VBox([btn_head_par_loan,End_season, fee_num, Redeemed])
input_par = widgets.HBox([left_box, right_box])
display(input_par)


## To perform exchanges run cells below AFTER selecting parameters

to avoid cross dependence the best way is to run the cell after parameters are selected.
It applies to this only

In [19]:
list_ow = owner_players(new_owner)
list_ow.insert(0,'Choose one')
Ex_players = widgets.Dropdown(options = list_ow, description = 'Exch. Player')

def Ex_pl_eventhandler(change):
    
    global exch_pl
    exch_pl = change.new
    print('If exchange is chosen, '+pl.name+' will be exchanged with ' + exch_pl+'              ', end="\r", flush=True)
    
Ex_players.observe(Ex_pl_eventhandler, names = 'value')

#display(Ex_players)

Ex_squads = widgets.Dropdown(options = ['Choose one','Main', 'Primavera'], description = 'Exch. Squad')

def squad_eventhandler(change):
    global squad_ex_pl
    squad_ex_pl = change.new
    print('After operation '+exch_pl+' will be in ' + G_owner+'\'s '+squad_ex_pl+' squad                ', end="\r", flush=True)

Ex_squads.observe(squad_eventhandler, names = 'value')

#display(Ex_players)
#display(Ex_squads)

btn_head_exch = widgets.Button(description='Select Exchange Player (RUN CELL FIRST)',layout=widgets.Layout(width='350px', height='40px'))

btn_head_exch.style.button_color = 'skyblue'


input_ex = widgets.VBox([btn_head_exch,Ex_players,Ex_squads])
display(input_ex)


## Remember: Confirm parameters before performing operation

In [61]:
btn_confirm_parameters = widgets.Button(description='Confirm Parameters',layout=widgets.Layout(width='603px', height='40px'))

def confirm_parameters_event(obj):
    
    btn_draft = widgets.Button(description='Draft Player')
    
    def draft_event(obj): 
        
        check = pl.draft_in(new_owner, squad = squad, cost = cost)
        clear_output()
        if check:
            print(pl.name + ' was drafted by '+ new_owner+'. The cost of the operation is '+str(cost)+' FM.               ', end="\r", flush=True)
        else:
            print('Unsufficient Funds!! Operation was canceled')
        display(btn_confirm_parameters)
    btn_draft.on_click(draft_event)

    btn_auction = widgets.Button(description='Auction Player')
    
    def auction_event(obj):
        
        pl.auction_in(new_owner, squad = squad, cost = cost)
        clear_output()
        print(pl.name + ' was adjudicated by '+ new_owner+'. The cost of the operation is '+str(cost)+' FM.                ', end="\r", flush=True)
        display(btn_confirm_parameters)
    btn_auction.on_click(auction_event)

    btn_cash = widgets.Button(description='Buy Player')
    
    def cash_event(obj):
        
        pl.cash_in(new_owner, squad = squad, cost = cost)
        clear_output()
        print(pl.name + ' was purchased by '+ new_owner+'. The cost of the operation is '+str(cost)+' FM.                    ', end="\r", flush=True)
        display(btn_confirm_parameters)
        
        
    btn_cash.on_click(cash_event)

    btn_exch = widgets.Button(description='Exchange Players')
    
    def exch_event(obj):
        if cost > 0:
            payer = new_owner
        else:
            payer = G_owner
        
        pl.exchange_with(exch_pl, cost, squad, squad_ex_pl)
        clear_output()
        print(pl.name + ' was exchanged with '+ exch_pl+'. '+payer+' paid '+str(abs(cost))+' FM.                         ', end="\r", flush=True)
        display(btn_confirm_parameters)
    btn_exch.on_click(exch_event)

    btn_loan = widgets.Button(description='Start Loan')
    
    def loan_event(obj):
        
        pl.loan_to(new_owner, cost, squad, end_season, final_fee)
        clear_output()
        print(pl.name + ' was sent on loan to '+ new_owner+'. The cost of the operation is '+str(cost)+' FM.                        ', end="\r", flush=True)
        display(btn_confirm_parameters)
    btn_loan.on_click(loan_event)


    btn_loan_end = widgets.Button(description='End Loan')
    
    def end_loan_event(obj):
        
        pl.end_loan(squad, redeemed)
        clear_output()
        if redeemed is True:
            print('Loan of '+pl.name + ' was terminated. Will be redeemed: Register separate transaction.                    ', end="\r", flush=True)
        else:
            print('Loan of '+pl.name + ' was terminated. Will NOT be redeemed.                         ', end="\r", flush=True)
        display(btn_confirm_parameters)
    btn_loan_end.on_click(end_loan_event)

    btn_move_up = widgets.Button(description='Move Up')
    
    def move_up_event(obj):
        
        pl.move_up()
        clear_output()
        print(pl.name + ' was moved to main squad.                                     ', end="\r", flush=True)
        display(btn_confirm_parameters)
    btn_move_up.on_click(move_up_event)
    
    
    #display(btn_draft)
    #display(btn_auction)
    #display(btn_cash)
    #display(btn_exch)
    #display(btn_loan)
    #display(btn_loan_end)
    #display(btn_move_up)
    
    btn_head_1pl = widgets.Button(description='1 pl Ops.')
    btn_head_2pl = widgets.Button(description='Exchanges')
    btn_head_loans = widgets.Button(description='Loans')
    btn_head_prom = widgets.Button(description='Promotions')
    
    btn_head_1pl.style.button_color = 'skyblue'
    btn_head_2pl.style.button_color = 'skyblue'
    btn_head_loans.style.button_color = 'skyblue'
    btn_head_prom.style.button_color = 'skyblue'  
    
    box1 = widgets.VBox([btn_head_1pl,btn_draft, btn_auction, btn_cash])
    box2 = widgets.VBox([btn_head_2pl,btn_exch])
    box3 = widgets.VBox([btn_head_loans,btn_loan, btn_loan_end])
    box4 = widgets.VBox([btn_head_prom,btn_move_up])
    input_final = widgets.HBox([box1, box2, box3, box4])
    display(input_final)
    
    
btn_confirm_parameters.style.button_color = 'lightgreen'   
btn_confirm_parameters.on_click(confirm_parameters_event) 

display(btn_confirm_parameters)

Unsufficient Funds!! Operation was canceled


Button(description='Confirm Parameters', layout=Layout(height='40px', width='603px'), style=ButtonStyle(button…